## Step1: Clean the demographic data

In [1]:
import pandas as pd

# Load population, education & income data
demographic = pd.read_excel('/Users/Olivia/Desktop/Ontario Open Data Crime Statistics.xlsx', sheet_name='population_education_income')

demographic.head()

,Geography,postalcode,Census families in private households,Female lone-parent families,"Female lone-parent families, proportion of census families (percent)",Lone-parent families,"Lone-parent families, proportion of census families (percent)",Male lone-parent families,"Male lone-parent families, proportion of census families (percent)",Total population,...,Post-secondary graduates aged 25 to 54,"Post-secondary graduates aged 25 to 54, proportion of population aged 25 to 54 (percent)",Prevalence of low income before tax in 2010 for economic families for income status (percent),Prevalence of persons aged 17 years and under living in low income economic families before tax in 2010 (percent),Total economic families for income status,Total persons aged 17 years and under living in economic families,Total population in private households,Unemployment rate 15 years and over (percent),Visible minority population,"Visible minority population, proportion of total population (percent)"
0,"Brant County Health Unit, Ontario [3527-A]",N3R 1G7,39470.0,5965,15.1,7450,18.9,1485,3.8,137100,...,32250,59.1,8.2,12.9,36745,30820,134850,7.6,7490,5.6
1,"Chatham-Kent Health Unit, Ontario [3540-E]",N7M 5L8,30415.0,3985,13.1,5055,16.6,1070,3.5,104075,...,20620,52.2,10.8,18.2,30130,21650,102075,10.2,4015,3.9
2,"City of Hamilton Health Unit, Ontario [3537-A]",L8P 4S6,144120.0,21925,15.2,27220,18.9,5290,3.7,519950,...,135625,63.9,14.0,22.6,141850,105955,509640,8.7,79970,15.7
3,"City of Toronto Health Unit, Ontario [3595-G]",M5B 1W2,690340.0,123530,17.9,146990,21.3,23455,3.4,2615060,...,849145,71.2,18.9,28.4,675900,486400,2576025,9.3,1264390,49.1
4,"District of Algoma Health Unit, Ontario [3526-E]",P6B 0A8,34300.0,4650,13.6,5955,17.4,1305,3.8,114785,...,26215,61.7,8.4,14.9,33280,20370,112750,10.9,1695,1.5


In [2]:
# Check the data type of individual columns, see if modification is required
demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 36 columns):
 #   Column                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                             --------------  -----  
 0   Geography                                                                                                          35 non-null     object 
 1   postalcode                                                                                                         35 non-null     object 
 2   Census families in private households                                                                              34 non-null     float64
 3   Female lone-parent families                                                                                        35 non-null     int64  
 4   Female lone-parent families,

In [3]:
# Check for missing values

# Find columns with null values
null_counts = demographic.isnull().sum()

# If a column contains null value(s), print the number of null values
for column, count in null_counts.items():
    if count > 0:
        print(f"Column '{column}' has {count} null values.")

Column 'Census families in private households' has 1 null values.


In [4]:
# Impute the missing values

# Calculate the percentage of 'Census families in private households' in 'Total Population'
demographic['Ratio'] = demographic['Census families in private households'] / demographic['Total population']

# Calculate the average percentage
average_ratio = demographic['Ratio'].mean()

# 3.Muptiple the 'Census families in private households' by the average percentage. Fill the empty cell with the estimated value
demographic['Census families in private households'].fillna((demographic['Total population'] * average_ratio).astype(int), inplace=True)

In [5]:
# Check for duplicate rows. If there are any duplicates, remove them.

# Detect duplicate rows
duplicates = demographic[demographic.duplicated(keep=False)]  

# Print duplicate rows
if not duplicates.empty:
    print("Duplicate Rows:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [6]:
# Define a function to shorten the column
def clean_region_name(record):
    return record.split(', Ontario')[0]

# Apply the function to the 'Geography' column 
demographic['Geography'] = demographic['Geography'].apply(clean_region_name)

# Display the updated column
demographic['Geography'].head()

0          Brant County Health Unit
1          Chatham-Kent Health Unit
2      City of Hamilton Health Unit
3       City of Toronto Health Unit
4    District of Algoma Health Unit
Name: Geography, dtype: object

In [7]:
# Drop the unecessary column
demographic = demographic.drop(columns=['Ratio'])

# Save the cleaned data to an Excel file
excel_file_path = '/Users/Olivia/Desktop/Demographic_Cleaned.xlsx'
demographic.to_excel(excel_file_path, sheet_name='Demographic Cleaned')

## Step 2: Clean the crime data

In [8]:
# Load the crime data

crime = pd.read_excel('/Users/Olivia/Desktop/Ontario Open Data Crime Statistics.xlsx', sheet_name='Criminal_Offences')

crime.head()

,Record ID,Link Geography,Census_Divison,OFFENCE GROUP,OFFENCE TYPE,Cases Received,% of All Cases Received,Cases Disposed,Avg Days to Disposition\n(for cases without bench warrants),Avg Appearances to Disposition\n(for cases without bench warrants),...,Cases Disposed Before Trial Date TOTAL,Cases Disposed Before Trial Date Withdrawn/ Stayed Before Trial,Cases Disposed Before Trial Date Guilty Plea Before Trial,Cases Disposed Before Trial Date Other Disposition Before Trial,Cases Disposed at Trial Without Trial TOTAL,Cases Disposed at Trial Without Trial Withdrawn at Trial,Cases Disposed at Trial Without Trial Guilty Plea at Trial,Cases Disposed at Trial Without Trial Other Disposition at Trial,Cases Disposed Following a Trial TOTAL,Trial Rate
0,1,"Simcoe Muskoka District Health Unit, Ontario [...",Barrie,Crimes Against the Person,Homicide,0,0.0%,2,325.0,16.5,...,0,0,0,0,0,0,0,0,2,100.0%
1,2,"Simcoe Muskoka District Health Unit, Ontario [...",Barrie,Crimes Against the Person,Attempted Murder,6,0.8%,3,62.0,7.0,...,3,3,0,0,0,0,0,0,0,0.0%
2,3,"Simcoe Muskoka District Health Unit, Ontario [...",Barrie,Crimes Against the Person,Robbery,19,2.6%,10,194.0,8.1,...,8,4,4,0,2,1,1,0,0,0.0%
3,4,"Simcoe Muskoka District Health Unit, Ontario [...",Barrie,Crimes Against the Person,Sexual Assault,7,0.9%,9,313.0,13.8,...,8,1,7,0,1,1,0,0,0,0.0%
4,5,"Simcoe Muskoka District Health Unit, Ontario [...",Barrie,Crimes Against the Person,Other Sexual Offences,9,1.2%,5,271.0,4.6,...,3,2,0,1,2,2,0,0,0,0.0%


In [9]:
# Check the data type of individual columns, see if modification is required
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 21 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Record ID                                                          1674 non-null   int64  
 1   Link Geography                                                     1674 non-null   object 
 2   Census_Divison                                                     1674 non-null   object 
 3   OFFENCE GROUP                                                      1674 non-null   object 
 4   OFFENCE TYPE                                                       1674 non-null   object 
 5   Cases Received                                                     1674 non-null   int64  
 6   % of All Cases Received                                            1674 non-null   object 
 7   Cases Disposed          

In [10]:
# Change the data type of selected column from object to numerical

crime['Trial Rate'] = pd.to_numeric(crime['Trial Rate'], errors ='coerce')
crime['% of All Cases Received'] = pd.to_numeric(crime['% of All Cases Received'], errors='coerce')

In [11]:
# Fill missing values with 0
crime_numeric = crime.select_dtypes(include=['float64', 'int64']).fillna(0)
crime.update(crime_numeric)

# Verify that there are no missing values left
print(crime.isna().sum())

Record ID                                                             0
Link Geography                                                        0
Census_Divison                                                        0
OFFENCE GROUP                                                         0
OFFENCE TYPE                                                          0
Cases Received                                                        0
% of All Cases Received                                               0
Cases Disposed                                                        0
Avg Days to Disposition\n(for cases without bench warrants)           0
Avg Appearances to Disposition\n(for cases without bench warrants)    0
Cases Pending                                                         0
Cases Disposed Before Trial Date TOTAL                                0
Cases Disposed Before Trial Date Withdrawn/ Stayed Before Trial       0
Cases Disposed Before Trial Date Guilty Plea Before Trial       

In [12]:
# Resolve the discrepencies in 'Link Geography' column, transform all records into the same format

def remove_after_ontario(record):
    return record.split(', Ontario')[0]

# Apply the function to the 'Link Geography' column
crime['Link Geography'] = crime['Link Geography'].apply(remove_after_ontario)

# Display the updated column
crime['Link Geography'].head()

0    Simcoe Muskoka District Health Unit
1    Simcoe Muskoka District Health Unit
2    Simcoe Muskoka District Health Unit
3    Simcoe Muskoka District Health Unit
4    Simcoe Muskoka District Health Unit
Name: Link Geography, dtype: object

In [13]:
# Change the variants to the same naming convention
crime['OFFENCE GROUP'] = crime['OFFENCE GROUP'].replace('Admin of Justice', 'Administration of Justice')

# Verify the changes by printing unique values in the 'Offense Group' column
print(crime['OFFENCE GROUP'].unique())

['Crimes Against the Person' 'Crimes Against Property'
 'Administration of Justice' 'Other Criminal Code' 'Criminal Code Traffic'
 'Federal Statute']


In [14]:
# Save the cleaned data to an Excel file
excel_file_path = '/Users/Olivia/Desktop/Criminal Offenses_Cleaned.xlsx'
crime.to_excel(excel_file_path, sheet_name='Criminal Offenses Cleaned')